In [1]:
import os

In [2]:
%pwd 

'h:\\Winequality-Predcition\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd 

'h:\\Winequality-Predcition'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path


In [6]:
from src.wine_quality.constants import *
from src.wine_quality.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformer_config =  DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path
        )

        return data_transformer_config

### Trasnformer Component

In [8]:
from src.wine_quality.logging import logger
from sklearn.model_selection import train_test_split as tts
import pandas as pd

In [9]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.model_selection import train_test_split as tts
 
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    # Shuffle the data
    def preprocess_data(self, data: pd.DataFrame) -> pd.DataFrame:
        data = data.sample(frac=1, random_state=42).reset_index(drop=True)
        logger.info("Data has been shuffled.")
        
        data = data.drop_duplicates()
        logger.info(f"Removed duplicates. Shape: {data.shape}")
        
        data = data.dropna()
        logger.info(f"Removed null values. Shape after removing nulls: {data.shape}")
        
        return data

    # MinMaxScaler
    def scale_data(self, data: pd.DataFrame) -> pd.DataFrame:
        scaler = MinMaxScaler()  
        numeric_columns = data.select_dtypes(include=["float64", "int64"]).columns
        data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
        logger.info("Data has been scaled using MinMaxScaler (range 0-1).")
        
        return data

    # Train-test split method
    def train_test_datasplit(self):
        data = pd.read_csv(self.config.data_path)
        
        # Preprocess and scale data
        data = self.preprocess_data(data)
        data = self.scale_data(data)

        # Split into train and test sets
        train, test = tts(data, test_size=0.2, random_state=42)
        
        # Save train and test data
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        # Logging the details
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")
        print(f"Train shape: {train.shape}, Test shape: {test.shape}")

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()  # Ensure this method exists and works
    data_transformation = DataTransformation(data_transformation_config)
    
    # Running the full pipeline
    data_transformation.train_test_datasplit()

except Exception as e:
    logger.error(f"Error occurred: {e}")
    raise e

[2024-09-22 12:58:51,294 : INFO : common : yaml file: config\config.yaml loaded successfully ]
[2024-09-22 12:58:51,298 : INFO : common : yaml file: params.yaml loaded successfully ]
[2024-09-22 12:58:51,301 : INFO : common : yaml file: schema.yaml loaded successfully ]
[2024-09-22 12:58:51,306 : INFO : common : created directory at: artifacts ]
[2024-09-22 12:58:51,308 : INFO : common : created directory at: artifacts/data_transformation ]
[2024-09-22 12:58:51,410 : INFO : 802715402 : Data has been shuffled. ]
[2024-09-22 12:58:51,521 : INFO : 802715402 : Removed duplicates. Shape: (1359, 12) ]
[2024-09-22 12:58:51,523 : INFO : 802715402 : Removed null values. Shape after removing nulls: (1359, 12) ]
[2024-09-22 12:58:51,593 : INFO : 802715402 : Data has been scaled using MinMaxScaler (range 0-1). ]
[2024-09-22 12:58:51,641 : INFO : 802715402 : Train shape: (1087, 12), Test shape: (272, 12) ]
Train shape: (1087, 12), Test shape: (272, 12)
